In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [18]:
import pandas as pd
import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import re
from keras.preprocessing.text import Tokenizer
from tqdm import tqdm
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score
# from joblib import dump, load
# from sklearn.naive_bayes import GaussianNB
# from google.colab import drive
# from xgboost import XGBClassifier
# from sklearn.svm import SVC
# from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, classification_report, mean_squared_error
# from sklearn.svm import SVC
# import matplotlib.pyplot as plt
# import seaborn as sns
# # from copy import copy
# from sklearn.ensemble import RandomForestClassifier
# # from catboost import CatBoostClassifier, Pool, cv
# from sklearn.metrics import accuracy_score
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Dense
# from sklearn.feature_selection import RFECV
import lightgbm as lgb
# from sklearn.svm import SVC
# from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def process_train(filepath):
    '''
    Reads file from filepath, returns a dataframe with required columns
    :param filepath: string path
    :param column_list: list of Dataframe column
    :return: Dataframe
    '''
    return pd.read_csv(filepath)

#Splitting train and test
def normalize_dataset(features_df):
    """
    Normalize the dataset
    :param features_dataframe:
    :return: normalized_features_array
    """ 
    #Using minmaxscaler from sklearn to normalize dataset
    min_max_scaler = MinMaxScaler()
    return min_max_scaler.fit_transform(features_df)

#Splitting train and test
def split_dataset(features_np_array, target_np_array, test_percentage, random_state):
    """
    Split the dataset with train_percentage
    :param dataset:
    :param test_percentage:
    :param feature_array:
    :param target_array:
    :return: train_x, test_x, train_y, test_y
    """ 
    #Split dataset into train and test dataset
    train_x, test_x, train_y, test_y = train_test_split(features_np_array, target_np_array, test_size = test_percentage, random_state = random_state)
    return train_x, test_x, train_y, test_y

def generate_csv(df,csv_name):
    '''
    Converting pred_labels to dataframe
    :param csv_name:
    :return: submission_csv
    '''
    return df.to_csv(csv_name, index = False)

def preprocess_title(s):
    '''
    '''
    if re.search(r'(.*)\/', s) is not None:
        k = re.search(r'(.*)\/', s).groups()[0].lower() 
        k = k.replace('[sound recording]', '').strip()
    elif re.findall(r'\w+', s) is not None:
        k = ' '.join(re.findall(r'\w+', s)).lower()
        k = k.replace('[sound recording]', '').strip()
    return k

def match_keywords(s):
    '''
    '''
    m = ['music', 'sound recording', 'musical']
    k = 0 
    for each in m:
        if each in s:
            k = 1
    else:
        k = 0
    return k

def creator_string(s):
    '''
    '''
    if s != 'None':
        s = ''.join(s.split(',')[0:2]).lower()
    return s



def creator_birth_year(s):
    '''
    '''
    k = re.findall(r'\d{4}', s)
    if len(k) != 0:
        try:
            k = sorted(k)[0]
            k = (pd.to_datetime('now').year - pd.to_datetime(k, format = '%Y').year)
        except:
            k = pd.period_range(start = k, end = '1678', freq = 'Y')
            k = len(k)
            k = (pd.to_datetime('now').year - pd.to_datetime('1678', format = '%Y').year + k)
    else:
        k = 0
    return k
    

def subject_tokens(s):
    '''
    '''
    if s != 'None':
        s = s.split(',')
        k = [re.sub(r'\d+', '', i).strip().lower() for i in s]
    else:
        k = []
    return k

def subject_min_year_token(s):
    '''
    '''
    k = re.findall(r'\d{4}', s)
    if len(k) != 0:
        try:
            k = sorted(k)[0]
            k = (pd.to_datetime('now').year - pd.to_datetime(k, format = '%Y').year)
        except:
            k = pd.period_range(start = k, end = '1678', freq = 'Y')
            k = len(k)
            k = (pd.to_datetime('now').year - pd.to_datetime('1678', format = '%Y').year + k)
    else:
        k = 0
    return k

def get_publisher(s):
    '''
    '''
    if re.search(r'(.*)\/', s) is not None:
        k = re.search(r'(.*)\/', s).groups()[0].lower()
    elif len(s.split(',')) != 0:
        k = ''.join(s.split(',')[0:2]).lower()
    return k

def publication_year(s):
    '''
    '''
    k = re.findall(r'\d{4}', s)
    if len(k) != 0:
        try:
            k = sorted(k)[0]
            k = (pd.to_datetime('now').year - pd.to_datetime(k, format = '%Y').year)
        except:
            k = pd.period_range(start = k, end = '1678', freq = 'Y')
            k = len(k)
            k = (pd.to_datetime('now').year - pd.to_datetime('1678', format = '%Y').year + k)
    else:
        k = 0
    return k



def split_dataset(features_np_array, target_np_array, test_percentage, random_state):
    """
    Split the dataset with train_percentage
    :param dataset:
    :param test_percentage:
    :param feature_array:
    :param target_array:
    :return: train_x, test_x, train_y, test_y
    """ 
    #Split dataset into train and test dataset
    train_x, test_x, train_y, test_y = train_test_split(features_np_array, target_np_array, test_size = test_percentage, random_state = random_state)
    return train_x, test_x, train_y, test_y
    
def read_glove(glove_path):
    '''

    :return:
    '''
    embeddings_index = dict()
    f = open(glove_path, encoding='utf8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index


def normalize_dataset(features_df):
    """
    Normalize the dataset
    :param features_dataframe:
    :return: normalized_features_array
    """ 
    #Using minmaxscaler from sklearn to normalize dataset
    min_max_scaler = MinMaxScaler()
    return min_max_scaler.fit_transform(features_df)


def prep_seq(col, word2vec_dict):
    '''
    '''
    traintest_X = []
    for sentence in tqdm(col.values):
        sequence_words = np.zeros((word2vec_dict['cucumber'].shape))
        for word in sentence:
            if word in word2vec_dict.keys():
                temp_X = word2vec_dict[word]
            else:
                temp_X = word2vec_dict['#']
            sequence_words+=(temp_X)/len(sentence)
        traintest_X.append(sequence_words)
    return np.array(traintest_X)
        

    
def label_to_mat(df_col):
    '''

    :param df_col:
    :return:
    '''
    le = LabelEncoder()
    le.fit(df_col.values)
    labels = le.transform(df_col.values)
    return to_categorical(labels, num_classes=len(le.classes_))


def mat_to_label(df_col, np_array):
    '''
    
    '''
    le = LabelEncoder()
    le.fit(df_col)
    return le.inverse_transform(np_array)



def generate_csv(df,csv_name):
    '''
    Converting pred_labels to dataframe
    :param csv_name:
    :return: submission_csv
    '''
    return df.to_csv(csv_name, index = False)
        
    
def job_title_map(s):
    '''
    '''
    if s != ' Anonymous Employee':
        s = 'None'
    return s
   
    
def summary_tokens(s):
    '''
    '''
    if s != 'None':
        k = re.split(r'\s|,', s)
        k = [j for j in k if len(j) != 0]
        k = [re.sub(r'\d+', '', i).strip().lower() for i in k]
        if len(k) > 5:
            k = k[:5]
        return k
    
def pos_neg_advice_tokens(s):
    '''
    '''
    if s != 'None' and type(s) is not list:
        k = re.split(r'\s|,|\/', s)
        k = [re.sub(r'\d+', '', i).strip().lower() for i in k]
        k = [i.strip('+~*\'"?!,.():;').lower() for i in k]
        k = sorted(set([j for j in k if len(j) != 0]), key = k.index)
        k = [j for j in k if j not in stop_words]
        if len(k) > 10:
            k = k[:10]
        return k
    
    
    
def merge_lists(l):
    # merge lists and return unique words as string
    tmp = []
    for i in l:
        if type(i) is list:
            tmp = tmp + i
    return tmp
            
         

In [0]:
train = process_train('/content/gdrive/My Drive/EricssonML/train_file.csv')
test = process_train('/content/gdrive/My Drive/EricssonML/test_file.csv')

In [0]:
# Preprocessing 

# Replacing NAN with 'None' string
train = train.replace(np.nan, 'None', regex = True)

# Mapping UsageClass feature values
train['UsageClass'] = train['UsageClass'].map({'Physical' : 0, 'Digital' : 1})

# Extracting total_year, total_month and total_days difference from CheckoutYear

train['CheckoutYear_year_diff'] = (pd.to_datetime('now') - pd.to_datetime(train['CheckoutYear'].astype(str) + '-0' + train['CheckoutMonth'].astype(str), format = '%Y-%M'))/np.timedelta64(1, 'Y')
train['CheckoutYear_month_diff'] = (pd.to_datetime('now') - pd.to_datetime(train['CheckoutYear'].astype(str) + '-0' + train['CheckoutMonth'].astype(str), format = '%Y-%M'))/np.timedelta64(1, 'M')
train['CheckoutYear_day_diff'] = (pd.to_datetime('now') - pd.to_datetime(train['CheckoutYear'].astype(str) + '-0' + train['CheckoutMonth'].astype(str), format = '%Y-%M'))/np.timedelta64(1, 'D')


# Extracting title tokens
train['Title_token'] = train['Title'].apply(lambda x : preprocess_title(x))

#
train['Creator_token'] = train['Creator'].apply(lambda x : creator_string(x))

#
train['Sound_recording_token'] = train['Title'].apply(lambda x : match_keywords(x))

#
train['Creator_birth_yr_diff'] = train['Creator'].apply(lambda x : creator_birth_year(x))

#
train['Subject_token'] = train['Subjects'].apply(lambda x : subject_tokens(x))

#
train['Publisher_token'] = train['Publisher'].apply(lambda x : get_publisher(x))

#
train['Publication_year_token'] = train['PublicationYear'].apply(lambda x : publication_year(x))

# Appending tokens
for i in range(len(train)):
    for j in ['Title_token', 'Publisher_token', 'Creator_token']:
        if train.loc[i, j].lower() != 'none':
            train.loc[i, 'Subject_token'].append(train.loc[i, j])
            
            
            
            


In [0]:
# Preprocessing 

# Replacing NAN with 'None' string
test = test.replace(np.nan, 'None', regex = True)

# Mapping UsageClass feature values
test['UsageClass'] = test['UsageClass'].map({'Physical' : 0, 'Digital' : 1})

# Extracting total_year, total_month and total_days difference from CheckoutYear

test['CheckoutYear_year_diff'] = (pd.to_datetime('now') - pd.to_datetime(test['CheckoutYear'].astype(str) + '-0' + test['CheckoutMonth'].astype(str), format = '%Y-%M'))/np.timedelta64(1, 'Y')
test['CheckoutYear_month_diff'] = (pd.to_datetime('now') - pd.to_datetime(test['CheckoutYear'].astype(str) + '-0' + test['CheckoutMonth'].astype(str), format = '%Y-%M'))/np.timedelta64(1, 'M')
test['CheckoutYear_day_diff'] = (pd.to_datetime('now') - pd.to_datetime(test['CheckoutYear'].astype(str) + '-0' + test['CheckoutMonth'].astype(str), format = '%Y-%M'))/np.timedelta64(1, 'D')


# Extracting title tokens
test['Title_token'] = test['Title'].apply(lambda x : preprocess_title(x))

#
test['Creator_token'] = test['Creator'].apply(lambda x : creator_string(x))

#
test['Sound_recording_token'] = test['Title'].apply(lambda x : match_keywords(x))

#
test['Creator_birth_yr_diff'] = test['Creator'].apply(lambda x : creator_birth_year(x))

#
test['Subject_token'] = test['Subjects'].apply(lambda x : subject_tokens(x))

#
test['Publisher_token'] = test['Publisher'].apply(lambda x : get_publisher(x))

#
test['Publication_year_token'] = test['PublicationYear'].apply(lambda x : publication_year(x))

# Appending tokens
for i in range(len(test)):
    for j in ['Title_token', 'Publisher_token', 'Creator_token']:
        if test.loc[i, j].lower() != 'none':
            test.loc[i, 'Subject_token'].append(test.loc[i, j])

In [0]:
combined_docs = train['Subject_token'].append(test['Subject_token'])

In [133]:
train['MaterialType'].value_counts()

BOOK         21707
SOUNDDISC     4149
VIDEOCASS     2751
VIDEODISC     1420
SOUNDCASS     1020
MIXED          347
MUSIC          165
CR              94
Name: MaterialType, dtype: int64

In [0]:

feature_columns = ['UsageClass', 'Checkouts', 'CheckoutYear_year_diff', 'CheckoutYear_month_diff', 'CheckoutYear_day_diff', 'Sound_recording_token', 'Creator_birth_yr_diff', 'Publication_year_token']
target_column = ['MaterialType']


features = train[feature_columns]
target = train[target_column]
target = target['MaterialType'].map({y:x for x, y in enumerate(train['MaterialType'].unique())})



train_features_array = normalize_dataset(features)


In [0]:

test_features_array = normalize_dataset(test[feature_columns])

In [10]:
# Using GLOVE word embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip /content/glove.6B.zip
embeddings_index = read_glove('/content/glove.6B.100d.txt')


--2019-05-29 07:04:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-05-29 07:04:35--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-05-29 07:04:35--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [91]:
# Preparing sequence for training

train_w2v = prep_seq(train['Subject_token'], embeddings_index)
test_w2v = prep_seq(test['Subject_token'], embeddings_index)

100%|██████████| 21102/21102 [00:00<00:00, 53510.51it/s]


In [0]:
train_features_array = np.concatenate([train_features_array, train_w2v], axis=-1)
test_features_array = np.concatenate([test_features_array, test_w2v], axis=-1)

In [0]:
train_X, val_X, train_Y, val_Y = split_dataset(train_features_array, target, 0.3, 42)

In [0]:
# Using cross-val for predictions
n_splits = 3
splits = list(StratifiedKFold(n_splits = n_splits, shuffle = True ,random_state = 1111).split(train_X, train_Y))

In [0]:
# Modelling
# Using LGBM
lgb_param = {
    'bagging_freq': 5,
        'bagging_fraction': 0.35,
        'boost_from_average':'false',
        'boost': 'gbdt',
        'feature_fraction': 0.0193,
        'learning_rate': 0.0291,
        'max_depth': -1,
        'metric':'multi_logloss',
        'min_data_in_leaf': 80,
        'min_sum_hessian_in_leaf': 12,
        'num_leaves': 8,    ### try 2; no interaction between variables makes sense here due to IID
        'tree_learner': 'serial',
        'objective': 'multiclass',
        'num_class':8,
        'verbosity': 1,
        "boost_from_average": "false"
}


In [0]:
oof = np.zeros((len(train_Y), 8))
predictions = np.zeros((len(test_features_array), 8))
for i, (train_idx, valid_idx) in enumerate(splits):
    
    x = np.array(train_X)
    y = np.array(train_Y)
    
    trn_data = lgb.Dataset(x[train_idx], label = y[train_idx])
    val_data = lgb.Dataset(x[valid_idx], label = y[valid_idx])
    
    model = lgb.train(lgb_param, trn_data, 10000, valid_sets = [trn_data, val_data], early_stopping_rounds=10000, verbose_eval=1000)
    
    oof[valid_idx] = model.predict(x[valid_idx], num_iteration = model.best_iteration)
        
#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["feature"] = features
#     fold_importance_df["importance"] = lgb_clf.feature_importance()
#     fold_importance_df["fold"] = i+1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += model.predict(test_features_array, num_iteration = model.best_iteration) / n_splits

print("Classification report".format(classification_report(train_Y, oof)))

In [0]:
le = LabelEncoder()
target = le.fit_transform(target)
 

In [0]:
predictions = le.inverse_transform(np.argmax(predictions, axis = 1))

In [0]:
cv_lgbm_pred_test = pd.DataFrame(predictions)
test_uid_df = pd.DataFrame(test['ID'], columns = ['ID'])
cv_lgbm_pred_test = test_uid_df.join(cv_lgbm_pred_test, how = 'inner')
cv_lgbm_pred_test.columns = ['ID', 'MaterialType']

In [135]:
cv_lgbm_pred_test['MaterialType'].value_counts()

0    20361
1      634
3       99
2        8
Name: MaterialType, dtype: int64

In [0]:
cv_lgbm_pred_test['MaterialType'] = cv_lgbm_pred_test['MaterialType'].map({x:y for x, y in enumerate(train['MaterialType'].unique())})

In [0]:
cv_lgbm_pred_test['MaterialType'].value_counts()
generate_csv(cv_lgbm_pred_test, 'cv_lgbm_pred_test.csv')

In [0]:
# Rating problem

train_rat = process_train('/content/gdrive/My Drive/EricssonML/Ratings/train.csv')
test_rat = process_train('/content/gdrive/My Drive/EricssonML/Ratings/test.csv')


In [0]:
# Preprocessing

# Mapping Place feature values
train_rat['Place'] = train_rat['Place'].map({y:x for x, y in enumerate(train_rat['Place'].unique())})

# Extracting month, year and days from review date

train_rat['Date_year_diff'] = (pd.to_datetime('now') - pd.to_datetime(train_rat['date'].apply(lambda x : x.strip()).astype(str), errors = 'coerce', format = '%b %d, %Y'))/np.timedelta64(1, 'Y')

train_rat['Date_month_diff'] = (pd.to_datetime('now') - pd.to_datetime(train_rat['date'].apply(lambda x : x.strip()).astype(str), errors = 'coerce', format = '%b %d, %Y'))/np.timedelta64(1, 'M')

train_rat['Date_day_diff'] = (pd.to_datetime('now') - pd.to_datetime(train_rat['date'].apply(lambda x : x.strip()).astype(str), errors = 'coerce', format = '%b %d, %Y'))/np.timedelta64(1, 'D')

train_rat['Date_year_diff'] = train_rat['Date_year_diff'].replace(np.nan, 0, regex = True)

train_rat['Date_month_diff'] = train_rat['Date_month_diff'].replace(np.nan, 0, regex = True)

train_rat['Date_day_diff'] = train_rat['Date_day_diff'].replace(np.nan, 0, regex = True)

# Mapping status feature values
train_rat['status'] = train_rat['status'].map({y:x for x, y in enumerate(train_rat['status'].unique())})

# Mapping values for job_title
train_rat['job_title'] = train_rat['job_title'].apply(lambda x: job_title_map(x)).map({' Anonymous Employee' : 0, 'None' : 1})

#
train_rat['summary'] = train_rat['summary'].replace(np.nan, 'None', regex = True)
train_rat['summary_tokens'] = train_rat['summary'].apply(lambda x : pos_neg_advice_tokens(x))
 
#
train_rat['positives'] = train_rat['positives'].replace(np.nan, 'None', regex = True)
train_rat['pos_tokens'] = train_rat['positives'].apply(lambda x : pos_neg_advice_tokens(x))

#
train_rat['negatives'] = train_rat['negatives'].replace(np.nan, 'None', regex = True)
train_rat['neg_tokens'] = train_rat['negatives'].apply(lambda x : pos_neg_advice_tokens(x))

#
train_rat['advice_to_mgmt'] = train_rat['advice_to_mgmt'].replace(np.nan, 'None', regex = True)
train_rat['advice_tokens'] = train_rat['advice_to_mgmt'].apply(lambda x : pos_neg_advice_tokens(x))
train_rat['advice_tokens'].loc[train_rat['advice_tokens'].isnull()] = train_rat['advice_tokens'].loc[train_rat['advice_tokens'].isnull()].apply(lambda x : [])


#
train_rat['tokens'] = train_rat['pos_tokens'] + train_rat['neg_tokens'] + train_rat['advice_tokens']

#
train_rat['overall'] = train_rat['overall'].astype('object')

In [0]:
# Preprocessing

# Mapping Place feature values
test_rat['Place'] = test_rat['Place'].map({y:x for x, y in enumerate(test_rat['Place'].unique())})

# Extracting month, year and days from review date

test_rat['Date_year_diff'] = (pd.to_datetime('now') - pd.to_datetime(test_rat['date'].apply(lambda x : x.strip()).astype(str), errors = 'coerce', format = '%b %d, %Y'))/np.timedelta64(1, 'Y')

test_rat['Date_month_diff'] = (pd.to_datetime('now') - pd.to_datetime(test_rat['date'].apply(lambda x : x.strip()).astype(str), errors = 'coerce', format = '%b %d, %Y'))/np.timedelta64(1, 'M')

test_rat['Date_day_diff'] = (pd.to_datetime('now') - pd.to_datetime(test_rat['date'].apply(lambda x : x.strip()).astype(str), errors = 'coerce', format = '%b %d, %Y'))/np.timedelta64(1, 'D')

test_rat['Date_year_diff'] = test_rat['Date_year_diff'].replace(np.nan, 0, regex = True)

test_rat['Date_month_diff'] = test_rat['Date_month_diff'].replace(np.nan, 0, regex = True)

test_rat['Date_day_diff'] = test_rat['Date_day_diff'].replace(np.nan, 0, regex = True)

# Mapping status feature values
test_rat['status'] = test_rat['status'].map({y:x for x, y in enumerate(test_rat['status'].unique())})

# Mapping values for job_title
test_rat['job_title'] = test_rat['job_title'].apply(lambda x: job_title_map(x)).map({' Anonymous Employee' : 0, 'None' : 1})

#
test_rat['summary'] = test_rat['summary'].replace(np.nan, 'None', regex = True)
test_rat['summary_tokens'] = test_rat['summary'].apply(lambda x : pos_neg_advice_tokens(x))
 
#
test_rat['positives'] = test_rat['positives'].replace(np.nan, 'None', regex = True)
test_rat['pos_tokens'] = test_rat['positives'].apply(lambda x : pos_neg_advice_tokens(x))

#
test_rat['negatives'] = test_rat['negatives'].replace(np.nan, 'None', regex = True)
test_rat['neg_tokens'] = test_rat['negatives'].apply(lambda x : pos_neg_advice_tokens(x))
test_rat['neg_tokens'].loc[test_rat['neg_tokens'].isnull()] = test_rat['neg_tokens'].loc[test_rat['neg_tokens'].isnull()].apply(lambda x : [])


#
test_rat['advice_to_mgmt'] = test_rat['advice_to_mgmt'].replace(np.nan, 'None', regex = True)
test_rat['advice_tokens'] = test_rat['advice_to_mgmt'].apply(lambda x : pos_neg_advice_tokens(x))
test_rat['advice_tokens'].loc[test_rat['advice_tokens'].isnull()] = test_rat['advice_tokens'].loc[test_rat['advice_tokens'].isnull()].apply(lambda x : [])


#
test_rat['tokens'] = test_rat['pos_tokens'] + test_rat['neg_tokens'] + test_rat['advice_tokens']




In [53]:
feature_columns = ['Place', 'Date_year_diff', 'Date_month_diff', 'Date_day_diff', 'status', 'job_title']
target_column = ['overall']

features = train_rat[feature_columns]
target = train_rat[target_column]


train_features_array = normalize_dataset(features)
test_features_array = normalize_dataset(test_rat[feature_columns])

le = LabelEncoder()
target = le.fit_transform(target)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [54]:
train_w2v = prep_seq(train_rat['tokens'], embeddings_index)
test_w2v = prep_seq(test_rat['tokens'], embeddings_index)

100%|██████████| 29272/29272 [00:02<00:00, 14635.48it/s]


In [0]:
train_features_array = np.concatenate([train_features_array, train_w2v], axis=-1)
test_features_array = np.concatenate([test_features_array, test_w2v], axis=-1)

In [0]:
train_X, val_X, train_Y, val_Y = split_dataset(train_features_array, target, 0.3, 42)

In [0]:
# Using cross-val for predictions
n_splits = 3
splits = list(StratifiedKFold(n_splits = n_splits, shuffle = True ,random_state = 1111).split(train_X, train_Y))

In [0]:
# Modelling
# Using LGBM
lgb_param = {
    'bagging_freq': 5,
        'bagging_fraction': 0.35,
        'boost_from_average':'false',
        'boost': 'gbdt',
        'feature_fraction': 0.0193,
        'learning_rate': 0.0291,
        'max_depth': -1,
        'metric':'multi_error',
        'min_data_in_leaf': 80,
        'min_sum_hessian_in_leaf': 12,
        'num_leaves': 8,    ### try 2; no interaction between variables makes sense here due to IID
        'tree_learner': 'serial',
        'objective': 'multiclass',
        'num_class':5,
        'verbosity': 1,
        "boost_from_average": "false"
}

In [0]:

oof = np.zeros((len(train_Y), 5))
predictions = np.zeros((len(test_features_array), 5))
for i, (train_idx, valid_idx) in enumerate(splits):
    
    x = np.array(train_X)
    y = np.array(train_Y)
    
    trn_data = lgb.Dataset(x[train_idx], label = y[train_idx])
    val_data = lgb.Dataset(x[valid_idx], label = y[valid_idx])
    
    model = lgb.train(lgb_param, trn_data, 10000, valid_sets = [trn_data, val_data], early_stopping_rounds=10000, verbose_eval=1000)
    
    oof[valid_idx] = model.predict(x[valid_idx], num_iteration = model.best_iteration)
        
#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["feature"] = features
#     fold_importance_df["importance"] = lgb_clf.feature_importance()
#     fold_importance_df["fold"] = i+1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += model.predict(test_features_array, num_iteration = model.best_iteration) / n_splits



In [0]:
predictions = le.inverse_transform(np.argmax(predictions, axis = 1))

In [0]:
cv_lgbm_pred_test = pd.DataFrame(predictions)
test_uid_df = pd.DataFrame(test_rat['ID'], columns = ['ID'])
cv_lgbm_pred_test = test_uid_df.join(cv_lgbm_pred_test, how = 'inner')
cv_lgbm_pred_test.columns = ['ID', 'overall']

In [0]:
cv_lgbm_pred_test['overall'].value_counts()

In [0]:
generate_csv(cv_lgbm_pred_test, 'cv_lgbm_pred_test.csv')